#### mp3 -> wav로 변환

In [2]:
#mp3 -> wav
!pip install pydub
# ffmpeg는 python 버전으로 conda 환경 설치하기

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6089 sha256=2f0d89c41f9cf4e01eeb233cdcdfb19f96399d041cb780e65cf31bb8d191795d
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\56\30\c5\576bdd729f3bc062d62a551be7fefd6ed2f761901568171e4e
Successfully built ffmpeg


In [14]:
# 앞에 10초 자르고 mp3->wav로 바꾸기
import os
from pydub import AudioSegment

# mp3 파일이 있는 디렉토리 경로
input_directory = "./mp3_emergency/"
# 변환된 wav 파일을 저장할 디렉토리 경로
output_directory = "./wav_emergency/"
# 잘라낼 앞부분 시간 (밀리초 단위)
trim_time = 10 * 1000  # 10초

# 출력 디렉토리가 없으면 생성
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 입력 디렉토리 내 모든 파일을 확인
for filename in os.listdir(input_directory):
    if filename.endswith(".mp3"):
        # mp3 파일의 전체 경로
        mp3_file_path = os.path.join(input_directory, filename)
        # wav 파일로 저장할 경로
        wav_file_path = os.path.join(output_directory, os.path.splitext(filename)[0] + ".wav")
        
        # mp3 파일을 AudioSegment 객체로 불러오기
        audio = AudioSegment.from_file(mp3_file_path, format="mp3")
        
        # 앞부분 10초를 잘라내기
        trimmed_audio = audio[trim_time:]
        
        # AudioSegment 객체를 wav 파일로 저장
        trimmed_audio.export(wav_file_path, format="wav")
        
        print(f"변환 완료: {filename} -> {os.path.splitext(filename)[0]}.wav")

print("모든 변환이 완료되었습니다!")

변환 완료: S-210915_S_101_D_002_0001.mp3 -> S-210915_S_101_D_002_0001.wav
변환 완료: S-211007_S_101_D_001_0001.mp3 -> S-211007_S_101_D_001_0001.wav
변환 완료: S-211014_S_101_C_003_0001.mp3 -> S-211014_S_101_C_003_0001.wav
변환 완료: S-211014_S_101_C_004_0001.mp3 -> S-211014_S_101_C_004_0001.wav
변환 완료: S-211020_S_101_C_001_0001.mp3 -> S-211020_S_101_C_001_0001.wav
변환 완료: S-211024_S_101_C_001_0001.mp3 -> S-211024_S_101_C_001_0001.wav
변환 완료: S-211024_S_101_C_001_0002.mp3 -> S-211024_S_101_C_001_0002.wav
변환 완료: S-211024_S_101_C_002_0001.mp3 -> S-211024_S_101_C_002_0001.wav
변환 완료: S-211024_S_101_C_014_0001.mp3 -> S-211024_S_101_C_014_0001.wav
변환 완료: S-211024_S_101_C_022_0001.mp3 -> S-211024_S_101_C_022_0001.wav
변환 완료: S-211024_S_101_C_035_0001.mp3 -> S-211024_S_101_C_035_0001.wav
변환 완료: S-211024_S_101_C_038_0001.mp3 -> S-211024_S_101_C_038_0001.wav
변환 완료: S-211024_S_101_C_041_0001.mp3 -> S-211024_S_101_C_041_0001.wav
변환 완료: S-211024_S_101_C_045_0001.mp3 -> S-211024_S_101_C_045_0001.wav
변환 완료: S-211024_S_10

####  전처리

In [19]:
import os
import librosa
import librosa.display
import numpy as np
import wave
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav

# 그래프에 - 표시
plt.rcParams['axes.unicode_minus'] = False

# 오디오 파일이 저장된 폴더 경로
folder_path = './wav_emergency/'  # 오디오 파일들이 있는 폴더 경로
output_folder = './result_emergency'  # 모든 결과 이미지를 저장할 폴더 경로
    
# 결과 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 최대 100개의 파일 처리
files = [f for f in os.listdir(folder_path) if f.endswith('.wav')][:40]

# 폴더 내의 모든 파일을 반복 처리
for filename in files:
    file_path = os.path.join(folder_path, filename)

    # 파일 정보 출력
    obj = wave.open(file_path, 'r')
    print(f"Processing {os.path.basename(file_path)}")
    print("Number of channels:", obj.getnchannels())
    print("Sampling rate:", obj.getframerate())
    print("Number of samples:", obj.getnframes())
    print("Play time(s):", obj.getnframes() / obj.getframerate())
    obj.close()

    # WAV 파일 읽기
    rate, data = wav.read(file_path)

    # 데이터를 부동 소수점으로 변환
    data = data.astype(np.float32)

    # 스테레오인 경우 모노로 변환
    if len(data.shape) > 1:
        data = librosa.to_mono(data.T)

    # 오디오 파일 자르기
    duration = data.shape[0] / rate  # 파일의 총 길이(초)
    chunk_size = 4 * rate  # 4초에 해당하는 샘플 수
    num_chunks = int(np.ceil(duration / 4))  # 총 몇 개의 4초 조각이 있는지

    for i in range(num_chunks):
        start_sample = i * chunk_size
        end_sample = min((i + 1) * chunk_size, data.shape[0])
        chunk_data = data[start_sample:end_sample]

        # 시간 축 생성 (0초에서 4초로 설정)
        time_axis = np.arange(start_sample, end_sample) / rate
        time_axis = time_axis - time_axis[0]  # 시작 시간을 0초로 맞춤


        # Librosa를 사용하여 오디오 청크 처리
        y = chunk_data

        # MFCC 계산 (40개 계수)
        mfcc_y = librosa.feature.mfcc(y=y, sr=rate, n_mfcc=40, n_fft=1024, hop_length=512)
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(mfcc_y, sr=rate, x_axis='time', hop_length=512)
        plt.colorbar()
        plt.title(f'MFCC (40) - {os.path.basename(file_path)} - Chunk {i + 1}')
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f'{os.path.splitext(filename)[0]}_mfcc_40_chunk_{i + 1}.png'))
        plt.close()

Processing S-210915_S_101_D_002_0001.wav
Number of channels: 1
Sampling rate: 44100
Number of samples: 1249000
Play time(s): 28.32199546485261
Processing S-211007_S_101_D_001_0001.wav
Number of channels: 1
Sampling rate: 44100
Number of samples: 249032
Play time(s): 5.646984126984127
Processing S-211014_S_101_C_003_0001.wav
Number of channels: 1
Sampling rate: 44100
Number of samples: 879177
Play time(s): 19.935986394557823
Processing S-211014_S_101_C_004_0001.wav
Number of channels: 1
Sampling rate: 44100
Number of samples: 813512
Play time(s): 18.446984126984127
Processing S-211020_S_101_C_001_0001.wav
Number of channels: 1
Sampling rate: 44100
Number of samples: 725974
Play time(s): 16.461995464852606
Processing S-211024_S_101_C_001_0001.wav
Number of channels: 1
Sampling rate: 44100
Number of samples: 668379
Play time(s): 15.155986394557823
Processing S-211024_S_101_C_001_0002.wav
Number of channels: 1
Sampling rate: 44100
Number of samples: 952912
Play time(s): 21.60798185941043
P

#### CNN

In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# 이미지 파일이 저장된 폴더 경로
folder_path = './result_emergency'  # 이미지 파일들이 있는 폴더 경로

# 데이터와 레이블을 저장할 리스트
X = []
y = []

# 폴더 내의 모든 파일을 반복 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.png'):
        file_path = os.path.join(folder_path, filename)
        # 이미지 로드 및 전처리
        image = load_img(file_path, color_mode='rgb', target_size=(128, 128))
        image = img_to_array(image)
        X.append(image)
        label = filename.split('_')[0]  # 파일명에서 레이블 추출 (예: '6.가전_5762_1.png'에서 '6.가전')
        y.append(label)

# 리스트를 NumPy 배열로 변환
X = np.array(X)
y = np.array(y)

# 레이블 인코딩
le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화 (0-255 범위의 픽셀 값을 0-1 범위로 변환)
X_train = X_train / 255.0
X_test = X_test / 255.0


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# 입력 데이터의 형태
input_shape = (128, 128, 3)  # 128x128 RGB 이미지

# 모델 구성
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(le.classes_), activation='softmax'))

# 모델 컴파일
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 63, 63, 32)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 30, 30, 64)       0         
 g2D)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 30, 30, 64)       

In [22]:
# 모델 학습
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
5/5 [==============================] - 4s 641ms/step - loss: 2.0769 - accuracy: 0.7926 - val_loss: 1.1329 - val_accuracy: 0.7941
Epoch 2/50
5/5 [==============================] - 2s 364ms/step - loss: 0.9751 - accuracy: 0.7852 - val_loss: 1.2059 - val_accuracy: 0.7941
Epoch 3/50
5/5 [==============================] - 2s 367ms/step - loss: 0.6625 - accuracy: 0.8667 - val_loss: 0.7986 - val_accuracy: 0.7941
Epoch 4/50
5/5 [==============================] - 2s 370ms/step - loss: 0.6946 - accuracy: 0.8667 - val_loss: 1.0407 - val_accuracy: 0.7941
Epoch 5/50
5/5 [==============================] - 2s 372ms/step - loss: 0.7030 - accuracy: 0.8667 - val_loss: 1.1504 - val_accuracy: 0.7941
Epoch 6/50
5/5 [==============================] - 2s 370ms/step - loss: 0.7270 - accuracy: 0.8667 - val_loss: 0.8692 - val_accuracy: 0.7941
Epoch 7/50
5/5 [==============================] - 2s 368ms/step - loss: 0.5911 - accuracy: 0.8667 - val_loss: 0.8357 - val_accuracy: 0.7941
Epoch 8/50
5/5 [====

In [ ]:
# 1분 소요

In [23]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

2/2 [==============================] - 0s 16ms/step - loss: 0.3680 - accuracy: 0.8529
Test Loss: 0.36801889538764954
Test Accuracy: 0.8529411554336548


In [ ]:
# mfcc가 30일 때 정확도 -> 0.7941176
# mfcc가 40일 때 정확도 -> 0.8529411